In [14]:
import torch
from torchvision import transforms
import torchvision.utils as vutils

from PIL import Image
import pandas as pd

import os

from tqdm import tqdm

import numpy as np

Notebook for generating new images for upsampling:

- Minority classes augmented using standard augmentations
- Minority classes augmented using AugMix
- Minority classes augmented using SMOTE
- Minority classes augmented using GAN (trial with4

# Upsampling via standard augmentations

In [2]:
# first need to get the csv data for simple upsampled data - to know which items have been duplicated
csv_path = './CSVs/train_simple_up.csv'
df_up = pd.read_csv(csv_path)
df_up.head()

id                                        imagename  cored   diffuse  \
0  10748           NA4757-02_AB/NA4757-02_AB_18_25_61.jpg    1.0  0.000000   
1  29503  NA4918-02_AB17-24/NA4918-02_AB17-24_9_18_12.jpg    0.0  2.832462   
2  42524  NA4885-02_AB17-24/NA4885-02_AB17-24_4_23_50.jpg    1.0  1.000000   
3  34432           NA4749-02_AB/NA4749-02_AB_17_12_50.jpg    0.0  3.770270   
4   4137            NA4751-02_AB/NA4751-02_AB_19_6_34.jpg    0.0  1.000000   

   CAA  negative  flag  notsure  
0  0.0       0.0   0.0      0.0  
1  0.0       0.0   0.0      0.0  
2  0.0       0.0   1.0      0.0  
3  0.0       0.0   0.0      0.0  
4  2.0       0.0   2.0      0.0

In [3]:
df_augment = df_up.loc[df_up.duplicated()==True,:]
df_augment = df_augment.sort_values(by='imagename', ignore_index=True).reset_index()
df_augment.head()

index     id                              imagename  cored  diffuse  CAA  \
0      0  14215  NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0      0.0  0.0   
1      1  14215  NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0      0.0  0.0   
2      2  14215  NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0      0.0  0.0   
3      3  14215  NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0      0.0  0.0   
4      4  14215  NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0      0.0  0.0   

   negative  flag  notsure  
0       0.0   0.0      0.0  
1       0.0   0.0      0.0  
2       0.0   0.0      0.0  
3       0.0   0.0      0.0  
4       0.0   0.0      0.0

In [4]:
# create name of images to save 
num_imgs = df_augment.shape[0]
img_names_aug = [f'std_aug/{num}.jpg' for num in range(num_imgs)]

In [5]:
df_augment['imagename_new'] = img_names_aug

In [6]:
df_augment

index     id                                          imagename  cored  \
0          0  14215              NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0   
1          1  14215              NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0   
2          2  14215              NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0   
3          3  14215              NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0   
4          4  14215              NA4009-02_AB/NA4009-02_AB_10_24_0.jpg    1.0   
...      ...    ...                                                ...    ...   
93864  93864  14846  NA_4888_02_AB17-24/NA_4888_02_AB17-24_8_26_15.jpg    1.0   
93865  93865  14846  NA_4888_02_AB17-24/NA_4888_02_AB17-24_8_26_15.jpg    1.0   
93866  93866  14846  NA_4888_02_AB17-24/NA_4888_02_AB17-24_8_26_15.jpg    1.0   
93867  93867  14846  NA_4888_02_AB17-24/NA_4888_02_AB17-24_8_26_15.jpg    1.0   
93868  93868  14846  NA_4888_02_AB17-24/NA_4888_02_AB17-24_8_26_15.jpg    1.0   

        diffuse  CAA  negative  flag  notsure      imagename_new  
0      0.000000  0.0       0.0   0.0      0.0      std_aug/0.jpg  
1      0.000000  0.0       0.0   0.0      0.0      std_aug/1.jpg  
2      0.000000  0.0       0.0   0.0      0.0      std_aug/2.jpg  
3      0.000000  0.0       0.0   0.0      0.0      std_aug/3.jpg  
4      0.000000  0.0       0.0   0.0      0.0      std_aug/4.jpg  
...         ...  ...       ...   ...      ...                ...  
93864  0.743902  0.0       0.0   0.0      0.0  std_aug/93864.jpg  
93865  0.743902  0.0       0.0   0.0      0.0  std_aug/93865.jpg  
93866  0.743902  0.0       0.0   0.0      0.0  std_aug/93866.jpg  
93867  0.743902  0.0       0.0   0.0      0.0  std_aug/93867.jpg  
93868  0.743902  0.0       0.0   0.0      0.0  std_aug/93868.jpg  

[93869 rows x 10 columns]

In [84]:
#transforms to apply:
trans_standard = transforms.Compose([
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomResizedCrop(256, scale=(0.8,1)),
                            transforms.RandomApply([transforms.RandomRotation((90, 90))], p=0.5),
                            transforms.ColorJitter(brightness=0.1, contrast=0.2,saturation=0.2, hue=0.02)])
#                             transforms.RandomAffine(0, translate=(0.05,0.05), scale=(0.9,1.1), shear=10)])

In [90]:
img_path = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val/'

for i, single_image_name in tqdm(enumerate(df_augment['imagename']), total=num_imgs):
#     print(single_image_name)
    img = Image.open(img_path + single_image_name)
    img = trans_standard(img)
    img.save(os.path.join(img_path, img_names_aug[i]), quality=95)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93869/93869 [22:13<00:00, 70.38it/s]


In [7]:
df_augment.reset_index(drop=True, inplace=True)

In [13]:
df_augment.drop(labels=['index', 'imagename'], axis=1, inplace=True)

In [22]:
df_augment.columns=['id', 'cored', 'diffuse', 'CAA', 'negative', 'flag', 'notsure', 'imagename']

In [23]:
df_augment= df_augment[['id','imagename','cored','diffuse','CAA','negative','flag','notsure']]

In [24]:
df_augment

id          imagename  cored   diffuse  CAA  negative  flag  notsure
0      14215      std_aug/0.jpg    1.0  0.000000  0.0       0.0   0.0      0.0
1      14215      std_aug/1.jpg    1.0  0.000000  0.0       0.0   0.0      0.0
2      14215      std_aug/2.jpg    1.0  0.000000  0.0       0.0   0.0      0.0
3      14215      std_aug/3.jpg    1.0  0.000000  0.0       0.0   0.0      0.0
4      14215      std_aug/4.jpg    1.0  0.000000  0.0       0.0   0.0      0.0
...      ...                ...    ...       ...  ...       ...   ...      ...
93864  14846  std_aug/93864.jpg    1.0  0.743902  0.0       0.0   0.0      0.0
93865  14846  std_aug/93865.jpg    1.0  0.743902  0.0       0.0   0.0      0.0
93866  14846  std_aug/93866.jpg    1.0  0.743902  0.0       0.0   0.0      0.0
93867  14846  std_aug/93867.jpg    1.0  0.743902  0.0       0.0   0.0      0.0
93868  14846  std_aug/93868.jpg    1.0  0.743902  0.0       0.0   0.0      0.0

[93869 rows x 8 columns]

In [25]:
df_augment.to_csv('./CSVs/train_stdaug_up.csv')

# AugMix

In [4]:
from augmix import augment_and_mix, augmentations

augmentations.IMAGE_SIZE=256

In [5]:
# create name of images to save 
num_imgs = df_augment.shape[0]
img_names_aug = [f'augmix/{num}.jpg' for num in range(num_imgs)]
df_augment['imagename_new'] = img_names_aug

In [94]:
# preprocess = transforms.Compose([ transforms.RandomHorizontalFlip(),
#                             transforms.RandomVerticalFlip(),
#                             transforms.RandomResizedCrop(256, scale=(0.8,1)),
#                             transforms.RandomApply([transforms.RandomRotation((90, 90))], p=0.5),
#                             transforms.ToTensor()])
trans = transforms.Compose([transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomResizedCrop(256, scale=(0.8,1)),
                            transforms.RandomApply([transforms.RandomRotation((90, 90))], p=0.5)])
preprocess = transforms.Compose([transforms.ToTensor()])

In [95]:
args = {'all_ops': True, 'mixture_width': 3, 'mixture_depth':-1, 'aug_severity': 3}

In [96]:
def aug(image, preprocess, args):
    """Perform AugMix augmentations and compute mixture.
    Args:
    image: PIL.Image input image
    preprocess: Preprocessing function which should return a torch tensor.
    Returns:
    mixed: Augmented and mixed image.
    """
    aug_list = augmentations.augmentations
    if args['all_ops']:
        aug_list = augmentations.augmentations_all

    ws = np.float32(np.random.dirichlet([1] * args['mixture_width']))
    m = np.float32(np.random.beta(1, 1))

    mix = torch.zeros_like(preprocess(image))
    for i in range(args['mixture_width']):
        image_aug = image.copy()
        depth = args['mixture_depth'] if args['mixture_depth'] > 0 else np.random.randint(
            1, 4)
        for _ in range(depth):
            op = np.random.choice(aug_list)
            image_aug = op(image_aug, args['aug_severity'])
        # Preprocessing commutes since all coefficients are convex
        mix += ws[i] * preprocess(image_aug)

    mixed = (1 - m) * preprocess(image) + m * mix
    return mixed

In [97]:
img_path = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val/'
pil_img = Image.open(img_path + df_augment.loc[0,'imagename'])
pil_img

In [131]:
mixed_img = aug(trans(pil_img), preprocess, args)

In [132]:
pil_mixed = transforms.ToPILImage()(mixed_img)

In [133]:
pil_mixed

In [135]:
img_names_aug

['augmix/0.jpg',
 'augmix/1.jpg',
 'augmix/2.jpg',
 'augmix/3.jpg',
 'augmix/4.jpg',
 'augmix/5.jpg',
 'augmix/6.jpg',
 'augmix/7.jpg',
 'augmix/8.jpg',
 'augmix/9.jpg',
 'augmix/10.jpg',
 'augmix/11.jpg',
 'augmix/12.jpg',
 'augmix/13.jpg',
 'augmix/14.jpg',
 'augmix/15.jpg',
 'augmix/16.jpg',
 'augmix/17.jpg',
 'augmix/18.jpg',
 'augmix/19.jpg',
 'augmix/20.jpg',
 'augmix/21.jpg',
 'augmix/22.jpg',
 'augmix/23.jpg',
 'augmix/24.jpg',
 'augmix/25.jpg',
 'augmix/26.jpg',
 'augmix/27.jpg',
 'augmix/28.jpg',
 'augmix/29.jpg',
 'augmix/30.jpg',
 'augmix/31.jpg',
 'augmix/32.jpg',
 'augmix/33.jpg',
 'augmix/34.jpg',
 'augmix/35.jpg',
 'augmix/36.jpg',
 'augmix/37.jpg',
 'augmix/38.jpg',
 'augmix/39.jpg',
 'augmix/40.jpg',
 'augmix/41.jpg',
 'augmix/42.jpg',
 'augmix/43.jpg',
 'augmix/44.jpg',
 'augmix/45.jpg',
 'augmix/46.jpg',
 'augmix/47.jpg',
 'augmix/48.jpg',
 'augmix/49.jpg',
 'augmix/50.jpg',
 'augmix/51.jpg',
 'augmix/52.jpg',
 'augmix/53.jpg',
 'augmix/54.jpg',
 'augmix/55.jpg',
 '

In [139]:
args = {'all_ops': True, 'mixture_width': 3, 'mixture_depth':-1, 'aug_severity': 3}
img_path = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val/'

trans = transforms.Compose([transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomResizedCrop(256, scale=(0.8,1)),
                            transforms.RandomApply([transforms.RandomRotation((90, 90))], p=0.5)])
preprocess = transforms.Compose([transforms.ToTensor()])

for i, single_image_name in tqdm(enumerate(df_augment['imagename']), total=num_imgs):
#     print(single_image_name)
    img = Image.open(img_path + single_image_name)
    img = aug(trans(img), preprocess, args)
    img = transforms.ToPILImage()(img)
    img.save(os.path.join(img_path, img_names_aug[i]), quality=95)

100%|████████████████████████████████████████████████████████████████████████████| 93869/93869 [36:17<00:00, 43.11it/s]


In [140]:
df_augment.reset_index(drop=True, inplace=True)
df_augment.drop(labels=['index', 'imagename'], axis=1, inplace=True)
df_augment.columns=['id', 'cored', 'diffuse', 'CAA', 'negative', 'flag', 'notsure', 'imagename']
df_augment= df_augment[['id','imagename','cored','diffuse','CAA','negative','flag','notsure']]
df_augment.to_csv('./CSVs/train_augmix_up.csv',index=False)

# SMOTE

In [4]:
# create name of images to save 
num_imgs = df_augment.shape[0]
img_names_aug = [f'smote/{num}.jpg' for num in range(num_imgs)]

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
# oversample each of the classes
# first need to get the csv data for the non-upsampled training data
csv_path = './CSVs/train.csv'
labels_df = pd.read_csv(csv_path)
# df.head()
morphologies = ["cored","diffuse","CAA"]
morphologies_filter = ["cored","CAA",["cored","diffuse"],["CAA","diffuse"], ["cored","CAA"]]

labels_df = labels_df[["imagename","cored","diffuse","CAA"]]
# labels_df["imagename"] = labels_df["imagename"].apply(os.path.basename)
# convert labels to 1s and 0s
labels_df[["cored","diffuse","CAA"]] = labels_df[["cored","diffuse","CAA"]].gt(0.99).astype(int)
labels_df.head()

imagename  cored  diffuse  CAA
0  NA4009-02_AB/neg_NA4009-02_AB_0_10_4.jpg      0        0    0
1  NA4009-02_AB/neg_NA4009-02_AB_0_11_2.jpg      0        0    0
2  NA4009-02_AB/neg_NA4009-02_AB_0_16_2.jpg      0        0    0
3  NA4009-02_AB/neg_NA4009-02_AB_0_25_2.jpg      0        0    0
4    NA4009-02_AB/NA4009-02_AB_10_10_14.jpg      0        1    0

In [7]:
img_path = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val/'
overall_array = []
for morphs in morphologies_filter:
    if type(morphs)==list:
        morph_name = '-'.join(morphs)
    else:
        morph_name = morphs
        morphs = [morphs]
    query_str = ''
    
    for morph in morphologies:
        if morph in morphs:
            query_str = query_str + morph + '==1 & '
        else:
            query_str = query_str + morph + '==0 & '

    query_str = query_str[:-3]
    temp_df = labels_df.query(query_str)
    temp_df.reset_index(drop=True, inplace=True)
#     print(temp_df.shape[0])
    temp_array = np.ndarray((temp_df.shape[0], 256 * 256 * 3), dtype='float16')
    for i, single_image_name in tqdm(enumerate(temp_df['imagename']), total=temp_df.shape[0]):
        img = Image.open(img_path + single_image_name)
        temp_array[i] = np.array(img).reshape(-1)/255
    overall_array.append(temp_array)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 254.40it/s]


In [56]:
overall_array = np.concatenate(overall_array, axis=0)

In [9]:
overall_y = [0*np.ones(1624,dtype='int'), 1*np.ones(1855,dtype='int'), 2*np.ones(509,dtype='int'), 3*np.ones(364,dtype='int'), 4*np.ones(7,dtype='int')]
# ys = np.concatenate(ys, axis=0)

In [135]:
target_gen = np.array([35728, 38955, 11198, 7644, 301])
num_gen = np.array([36000, 0, 100000, 100000, 100000])

a = np.where(num_gen<target_gen)[0]

In [138]:
a.shape[0]==1

True

In [139]:
a = np.insert(a,1,3)
a

array([1, 3], dtype=int64)

In [88]:
# number of each class to generate
target_gen = np.array([35728, 38955, 11198, 7644, 301])

# doing SMOTE in mini batches to speed up and keep memory low
sample_frac = np.array([0.1, 0.1, 0.5, 0.5, 1])

num_gen = np.array([0, 0, 0, 0, 0])



In [103]:
classes_sample = np.where(num_gen<target_gen)[0]

for c in classes_sample:
    print(c)

1
2
3


In [91]:
classes_sample= np.array([0, 2, 3])

for target, frac, gen in zip(target_gen[class2sample], sample_frac[class2sample], num_gen[class2sample]) for class2sample in classes_sample:
    print(f'{target},{frac},{gen}')

SyntaxError: invalid syntax (<ipython-input-91-8273a7534044>, line 3)

In [66]:
this_dict = {k: v for k,v in zip(list(range(5)), [100, 200, 300, 400, 500])}

In [69]:
map_dict = {0: [1, 0, 0], 1: [0, 0, 1], 2: [1, 1, 0], 3: [0, 1, 1], 4: [1, 0, 1]}

In [70]:
labels_stacked = []
labels = [0, 1 ,1 ,2, 0, 3, 2, 4]

for label in labels:
    labels_stacked.append(map_dict[label])

labels_stacked

[[1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 1, 0],
 [1, 0, 0],
 [0, 1, 1],
 [1, 1, 0],
 [1, 0, 1]]

In [128]:
a = np.array([0], dtype='int64')

In [129]:
a.shape

(1,)

In [73]:
paths_stacked = [f'{i}.png' for i in range(8)]

In [110]:
type(overall_y[0])

numpy.ndarray

In [144]:
def sample_smote(overall_array, overall_y, save_dir = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val'):
    # classes are:
    # cored: 0
    # caa:   1
    # cored + diffuse: 2
    # caa + diffuse: 3
    # cored + caa: 4
    
    # number of each class to generate
    target_gen = np.array([35728, 38955, 11198, 7644, 301])
    
    # doing SMOTE in mini batches to speed up and keep memory low
    sample_fracs = np.array([0.3, 0.3, 1, 1, 1])
    
    num_gen = np.array([0, 0, 0, 0, 0])
    morphs = [0,1,2,3,4,5]
    morphs_map = {0: [1, 0, 0], 1: [0, 0, 1], 2: [1, 1, 0], 3: [0, 1, 1], 4: [1, 0, 1]}
    
    save_counter = 0
    save_paths = []
    labels_stacked = []
    while np.any(num_gen < target_gen):
        print(f'Currently generated: {num_gen}')
        
        classes_sample = np.where(num_gen<target_gen)[0]
        classes_sample = classes_sample.astype(np.int64)
        if classes_sample.shape[0] == 1:
            classes_sample = np.insert(classes_sample,1,4)
        sampled_im_array = []
        sampled_y_array = []
        num_selects = []
        morphs_sample = []
        num_orig = 0
        for class2sample in classes_sample:
#             print(class2sample)
            im_array = overall_array[class2sample]
            y_array = overall_y[class2sample]
            sample_frac = sample_fracs[class2sample]
            morph = morphs[class2sample]
#             for im_array, y_array, sample_frac, morph in zip(overall_array[class2sample], overall_y[class2sample], sample_frac[class2sample], morphs[class2sample]):
            morphs_sample.append(morph)
            num_select = int(np.ceil(sample_frac*im_array.shape[0]))
            num_orig += num_select
            num_selects.append(num_select*2)
            idx_select = np.random.choice(im_array.shape[0], num_select, replace=False)
            sampled_im_array.append(im_array[idx_select])
            sampled_y_array.append(y_array[idx_select])
        
        strat = {k:v for k,v in zip(morphs_sample,num_selects)}
        sm = SMOTE(sampling_strategy=strat)
        
        # generate samples
        sampled_im_array = np.concatenate(sampled_im_array, axis=0)
        sampled_y_array = np.concatenate(sampled_y_array, axis=0)
        X_res, y_res = sm.fit_resample(sampled_im_array, sampled_y_array)
#         print(X_res.shape)
#         print(y_res.shape)
        # get rid of original training data!
        X_res = X_res[num_orig:]
        y_res = y_res[num_orig:]
#         print(X_res.shape)
#         print(y_res.shape)
        # reshape and save
        for img, label in zip(X_res, y_res):
            # reshape and save image
            if num_gen[label]< target_gen[label]:
                pil_img = Image.fromarray((img.reshape(256,256,3)*255).astype(np.uint8))
                img_name = 'smote/' + f'{save_counter}.jpg'
                pil_img.save(os.path.join(save_dir, img_name), quality=95)
                num_gen[label]+=1
                labels_stacked.append(morphs_map[label])
                save_paths.append(img_name)
                save_counter +=1
            else:
                continue
    
    return save_paths, labels_stacked

In [145]:
smote_paths, labels_stacked = sample_smote(overall_array, overall_y, save_dir = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val')

Currently generated: [0 0 0 0 0]


C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (976) in class 0 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (1114) in class 1 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (1018) in class 2 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (728) in class 3 will be larger than the number of samples in the 

Currently generated: [488 557 509 364   7]


C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (976) in class 0 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (1114) in class 1 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (1018) in class 2 will be larger than the number of samples in the majority class (class #1 -> 557)
  warnings.warn(
C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (728) in class 3 will be larger than the number of samples in the 

Currently generated: [ 976 1114 1018  728   14]


...

C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (976) in class 0 will be larger than the number of samples in the majority class (class #0 -> 488)
  warnings.warn(


Currently generated: [35136 38955 11198  7644   301]


C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (976) in class 0 will be larger than the number of samples in the majority class (class #0 -> 488)
  warnings.warn(


Currently generated: [35624 38955 11198  7644   301]


C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (976) in class 0 will be larger than the number of samples in the majority class (class #0 -> 488)
  warnings.warn(


In [147]:
smote_paths

['smote/0.jpg',
 'smote/1.jpg',
 'smote/2.jpg',
 'smote/3.jpg',
 'smote/4.jpg',
 'smote/5.jpg',
 'smote/6.jpg',
 'smote/7.jpg',
 'smote/8.jpg',
 'smote/9.jpg',
 'smote/10.jpg',
 'smote/11.jpg',
 'smote/12.jpg',
 'smote/13.jpg',
 'smote/14.jpg',
 'smote/15.jpg',
 'smote/16.jpg',
 'smote/17.jpg',
 'smote/18.jpg',
 'smote/19.jpg',
 'smote/20.jpg',
 'smote/21.jpg',
 'smote/22.jpg',
 'smote/23.jpg',
 'smote/24.jpg',
 'smote/25.jpg',
 'smote/26.jpg',
 'smote/27.jpg',
 'smote/28.jpg',
 'smote/29.jpg',
 'smote/30.jpg',
 'smote/31.jpg',
 'smote/32.jpg',
 'smote/33.jpg',
 'smote/34.jpg',
 'smote/35.jpg',
 'smote/36.jpg',
 'smote/37.jpg',
 'smote/38.jpg',
 'smote/39.jpg',
 'smote/40.jpg',
 'smote/41.jpg',
 'smote/42.jpg',
 'smote/43.jpg',
 'smote/44.jpg',
 'smote/45.jpg',
 'smote/46.jpg',
 'smote/47.jpg',
 'smote/48.jpg',
 'smote/49.jpg',
 'smote/50.jpg',
 'smote/51.jpg',
 'smote/52.jpg',
 'smote/53.jpg',
 'smote/54.jpg',
 'smote/55.jpg',
 'smote/56.jpg',
 'smote/57.jpg',
 'smote/58.jpg',
 'smote

In [150]:
df_out = pd.DataFrame(data=smote_paths,columns=['imagename'])
df_out[['cored','diffuse','CAA']] = labels_stacked
df_out.head()

imagename  cored  diffuse  CAA
0  smote/0.jpg      1        0    0
1  smote/1.jpg      1        0    0
2  smote/2.jpg      1        0    0
3  smote/3.jpg      1        0    0
4  smote/4.jpg      1        0    0

In [151]:
df_out.to_csv('./CSVs/train_smote_up.csv',index=False)

# Upsampling with GAN - Classifier Confidence

In [ ]:
def truncated_z_sample(batch_size, z_dim, truncation = 0.5, seed = None):
    state = None if seed is None else np.random.RandomState(seed)
    if truncation > 0:
        values = truncnorm.rvs(-2, 2, size=(batch_size, z_dim), random_state=state)
        return torch.as_tensor(truncation * values, dtype=torch.float32)
    else:
        return torch.randn((batch_size, z_dim))

def sample_generator(netG, z, norm_stats = None):
        # with amp.autocast():
        imgs = netG(z)[0].add(1).mul(0.5)
        # convert images from -1 1 to 0 255 uint8 (as would be done if saving)
        imgs = (imgs * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        # convert back to torch FloatTensor
        imgs = imgs.to(torch.float32).div_(255)
        # normalize
        if norm_stats is not None:
            # using mean and std of training set
            imgs = Batched_Normalize(imgs, norm['mean'], norm['std'])
        return imgs


In [ ]:
# backbone of the CNN model to load the model parameters into
class Net(nn.Module):

    def __init__(self, fc_nodes=512, num_classes=3, dropout=0.5):
        super(Net, self).__init__()
        
    def forward(self, x):
 
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x
    
# load plaquebox model
device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
pbox_ckpt_path = '../Plaquebox/plaquebox-paper-master/models/CNN_model_parameters.pkl'
norm_stats_path = '../Plaquebox/plaquebox-paper-master/utils/normalization.npy'
norm = np.load(norm_stats_path, allow_pickle=True).item()
pbox_cnn = torch.load(pbox_ckpt_path, map_location=lambda storage, loc: storage)
pbox_cnn = list(pbox_cnn.modules())[1]
pbox_cnn.to(device)
_=pbox_cnn.eval()

norm_stats_path = '../Plaquebox/plaquebox-paper-master/utils/normalization.npy'
norm = np.load(norm_stats_path, allow_pickle=True).item()
trans_cnn = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize(mean=norm['mean'],std=norm['std'])])

In [13]:
base_dir = 'D:/ucl_masters_data/project/fastGAN_experiments'
morphs = ['cored','diffuse','cored-diffuse','CAA_diffuse']
ckpts_use = [50000, 45000, 35000, 45000]
target_gen = [35728, 38955, 11198, 7644, 301]
args_dict = {'base_dir': base_dir, 'morph': morphs, 'ckpts_use': ckpts_use, 'target_gen': target_gen}



In [11]:
def sample_gan_class_confidence(args_dict, classifier, norm, save_dir = '../Plaquebox/plaquebox-paper-master/data/tiles/train_and_val'):
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    base_dir = args_dict['base_dir']
    
    for morph, ckpt_iter, target_gen in zip(args_dict['morph'], args_dict['ckpts_use'], args_dict['target_gen']):
        ckpt_path = os.path.join(args_dict['base_dir'], morph, 'models', f'all_{ckpt_iter}.pth')
        checkpoint= torch,load(ckpt_path)
        
         # load in the generator
        try:
            with open(os.path.join(base_dir, morph, 'args.txt'), mode='r') as f:
                args_train = json.load(f)
                model_config = args_train['model_config']
                model_config = get_config['model_configs.csv', model_config, type='model']
                noise_dim = model_config['nz']
                
            netG = Generator(
                    nz                  = model_config['nz'],
                    activation          = model_config['g_activation'],
                    chan_attn           = model_config['g_chan_attn'],
                    skip_conn           = model_config['g_skip_conn'],
                    spatial_attn        = model_config['g_spatial_attn'],
                    attn_layers         = model_config['g_attn_layers'],
                    conv_layers         = model_config['g_conv_layers'],
                    alternate_layers    = model_config['g_alternate_layers'],
                    anti_alias          = model_config['g_anti_alias'],
                    noise_inj           = model_config['g_noise_inj'],
                    multi_res_out       = model_config['g_multi_res_out'],
                    small_im_size       = model_config['g_small_im_size'],
                )

            print('all ok!')   
        
        except:
            noise_dim=256
            netG = GeneratorOld(ngf=64, nz=noise_dim, im_size=256)
            print('old generator loaded!')
        
        # load in parameters
        if ema:
            load_params(netG, checkpoint['g_ema'])
        else:
            load_params(netG, checkpoint['g'])
            
        generated = 0
        while generated < target_gen:
            images = []
            for j in range(batch_class // batch_gen):
                z = truncated_z_sample(batch_gen, noise_dim, truncation=trunc).to(device)
                images.append(sample_generator(netG, z, norm_stats=norm))
            images = torch.cat(images)
            if images.shape[1] == 1:
                images = images.repeat([1,3,1,1])
            
            # run through classifier
            outputs = classifier(images)
            

In [12]:
sample_gan_class_confidence(args_dict)

D:/ucl_masters_data/project/fastGAN_experiments\cored\models\all_50000.pth
D:/ucl_masters_data/project/fastGAN_experiments\diffuse\models\all_45000.pth
D:/ucl_masters_data/project/fastGAN_experiments\cored-diffuse\models\all_35000.pth
D:/ucl_masters_data/project/fastGAN_experiments\CAA_diffuse\models\all_45000.pth
